In [1]:
%matplotlib inline

In [2]:
import numpy as np
import tensorflow as tf

from batcher import Batcher
from metrics import acc012

In [3]:
np.random.seed(1337)

In [4]:
X_train = np.load('../data/bin/train/deal.npy')
y_train = np.load('../data/bin/train/tricks_spades.npy')

X_val = np.load('../data/bin/val/deal.npy')
y_val = np.load('../data/bin/val/tricks_spades.npy')
X_val = X_val.reshape((X_val.shape[0], -1))
y_val = y_val.reshape((y_val.shape[0], 1))

n_examples = X_train.shape[0]

X_train = X_train.reshape((n_examples, -1))
y_train = y_train.reshape((n_examples, 1))

n_dim = X_train.shape[1]
assert(n_dim == 208)

batch_size = 64
n_iterations = 500000
display_step = 1000

learning_rate = 0.001

l2_reg = 0.05

#n_hidden_units = [512, 64, 64, 64]
n_hidden_units = [1024, 128, 128, 128]

In [5]:
X = tf.placeholder(tf.float32, [n_dim, None])
Y = tf.placeholder(tf.float32, [1, None])

In [6]:
weights = {}
biases = {}
activations = {'a0': X}

dimensions = n_dim

for i, n_units in enumerate(n_hidden_units):
    layer = i + 1
    w_name = 'w%d' % layer
    b_name = 'b%d' % layer
    a_name = 'a%d' % layer
    weights[w_name] = tf.get_variable(w_name, shape=[n_units, dimensions], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
    dimensions = n_units
    biases[b_name] = tf.Variable(np.zeros((n_units, 1)), dtype=tf.float32, name=b_name)
    a_prev = activations['a%d' % (layer - 1)]
    activations[a_name] = tf.nn.relu(tf.add(tf.matmul(weights[w_name], a_prev), biases[b_name]))

w_out = tf.get_variable('w_out', shape=[1, dimensions], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
b_out = tf.Variable(np.zeros((1, 1)), dtype=tf.float32, name='b_out')
pred = tf.add(tf.matmul(w_out, activations['a%d' % len(n_hidden_units)]), b_out)

In [7]:
rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(pred, Y)))
cost_pred = tf.reduce_mean(tf.squared_difference(pred, Y))
cost_reg = (l2_reg / (2*batch_size)) * sum([tf.reduce_sum(tf.square(w)) for w in weights.values()])
cost = cost_pred + cost_reg

In [8]:
#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
#optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
#optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9).minimize(cost)

In [9]:
init = tf.global_variables_initializer()

In [ ]:
batch = Batcher(n_examples, batch_size)

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(n_iterations):
        x_batch, y_batch = batch.next_batch([X_train, y_train])
        
        sess.run(optimizer, feed_dict={X:x_batch.T, Y:y_batch.T})
        
        if iteration % display_step == 0:
            c = sess.run(cost, feed_dict={X: X_train.T, Y: y_train.T})
            rms_error = sess.run(rmse, feed_dict={X: X_train.T, Y: y_train.T})
            pred_train = sess.run(pred, feed_dict={X: X_train.T, Y: y_train.T})
            pred_val = sess.run(pred, feed_dict={X: X_val.T, Y: y_val.T})
            print('it={} cost={} rmse={}'.format(iteration, c, rms_error))
            print(acc012(y_train, pred_train.T))
            print(acc012(y_val, pred_val.T))
            
    predictions = sess.run(pred, feed_dict={X: X_train.T, Y: y_train.T})
        

it=0 cost=47.768531799316406 rmse=6.887997627258301
(0.010986249999999999, 0.038898750000000003, 0.090975)
(0.011050000000000001, 0.038429999999999999, 0.090950000000000003)
it=1000 cost=1.5425890684127808 rmse=1.1051326990127563
(0.35508125000000001, 0.82985874999999998, 0.97423249999999995)
(0.35466999999999999, 0.83086000000000004, 0.97419)
it=2000 cost=1.4899108409881592 rmse=1.0829869508743286
(0.36525249999999998, 0.83997875, 0.97616250000000004)
(0.36748999999999998, 0.83933999999999997, 0.97574000000000005)
it=3000 cost=1.325942039489746 rmse=1.0063517093658447
(0.390295, 0.86790750000000005, 0.98440000000000005)
(0.38896999999999998, 0.86643000000000003, 0.98416000000000003)
it=4000 cost=1.239858865737915 rmse=0.9645533561706543
(0.40594124999999998, 0.88306499999999999, 0.98775999999999997)
(0.40410000000000001, 0.88063999999999998, 0.98733000000000004)
it=5000 cost=1.2084367275238037 rmse=0.9501156806945801
(0.41016625000000001, 0.88830500000000001, 0.98895500000000003)
(0.4

it=47000 cost=0.6434693336486816 rmse=0.6745204925537109
(0.55098999999999998, 0.97183375000000005, 0.99915874999999998)
(0.53205999999999998, 0.96440999999999999, 0.99853999999999998)
it=48000 cost=0.6320829391479492 rmse=0.667529284954071
(0.55492249999999999, 0.97346500000000002, 0.99924625)
(0.53381999999999996, 0.96623000000000003, 0.99882000000000004)
it=49000 cost=0.6373777985572815 rmse=0.6730358004570007
(0.55047875000000002, 0.97251874999999999, 0.99921375000000001)
(0.53093999999999997, 0.96526999999999996, 0.99870999999999999)
it=50000 cost=0.6313238739967346 rmse=0.6699898838996887
(0.55453874999999997, 0.97283375000000005, 0.99911749999999999)
(0.53493000000000002, 0.96513000000000004, 0.99860000000000004)
it=51000 cost=0.6177537441253662 rmse=0.6611126661300659
(0.55954749999999998, 0.97460749999999996, 0.999255)
(0.53752999999999995, 0.96687000000000001, 0.99878999999999996)
it=52000 cost=0.6416696310043335 rmse=0.6802554726600647
(0.54890249999999996, 0.970330000000000

it=93000 cost=0.5228455066680908 rmse=0.6295832395553589
(0.58220499999999997, 0.98072124999999999, 0.999525)
(0.54920999999999998, 0.97096000000000005, 0.99885999999999997)
